# Activity Tracking with APOLLO

## CONTINUE NOTE
Take notes on activities on [Notion](https://www.notion.so/7cups/Window-activity-tracking-desktop-phone-99b27dc20ac2429ea7f5b49958ca4d2d#0655378b1b1e4ef2ab051290e26d4293) and commit changes in git

In [1]:
# To use 100% width!
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/var/folders/fk/sjtjy3d1437533rdvljmg56h0000gn/T/ipykernel_85917/485875232.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
import datetime

In [3]:
import os
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()
pd.set_option('display.max_colwidth', 0)
datadir = lambda filename: os.path.join('USER_DATA', filename)

In [4]:
df = pd.read_csv(datadir('apollo.csv'), on_bad_lines='skip', delimiter='\t')
df.Timestamp = pd.to_datetime(df.Timestamp)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94155 entries, 0 to 94154
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Timestamp  94155 non-null  datetime64[ns]
 1   Activity   94155 non-null  object        
 2   Output     94155 non-null  object        
 3   Database   94155 non-null  object        
 4   Module     94155 non-null  object        
dtypes: datetime64[ns](1), object(4)
memory usage: 3.6+ MB


In [5]:
import re

In [6]:
previewdf = lambda df: display(HTML(df.to_html()))
previewseries = lambda series: display(HTML(pd.DataFrame(series).to_html()))

In [7]:
df.Activity.unique().tolist()

['Knowledge Sync Deletion Bookmark',
 'Knowledge Sync Addition Window',
 'Application Intents',
 'Now Playing',
 'Bluetooth Connected',
 'App Media Usage',
 'Device Lock Status',
 'Portrait Topic',
 'Backlight Status',
 'Application Web Usage',
 'Portrait Entity',
 'Notification Usage',
 'Application Usage',
 'Device Discoverability',
 'Event Tombstone']

In [8]:
df.Activity.value_counts(dropna=False)

Portrait Topic                      19312
Portrait Entity                     13837
Application Web Usage               10732
Notification Usage                  10455
Application Usage                   10368
Device Discoverability              10002
Application Intents                 8888 
Device Lock Status                  3790 
Backlight Status                    3779 
Knowledge Sync Addition Window      964  
Event Tombstone                     634  
Knowledge Sync Deletion Bookmark    482  
App Media Usage                     463  
Bluetooth Connected                 339  
Now Playing                         110  
Name: Activity, dtype: int64

In [15]:
class Crawler:
    apollo_cols = ['Timestamp', 'Activity', 'Output', 'Database', 'Module']
    ts = 'Timestamp'
    adjusted_ts = 'AdjustedTimestamp'
    gmt_offset = 'GMT OFFSET'
    def __init__(self, df):
        self.row_count = 0
        self.df = df
        self.activities = df.Activity.unique().tolist()
        self.tables = {act: None for act in self.activities}
    
    def get_output_df(self, output_series):
        out = output_series
        out_dict = {}
        all_keys = set()
        out_len = 0
        for row in out:
            attributes = re.findall('\[+(.*?)\]', row)
            attr_dict = {}
            for attr in attributes:
                tmp = attr.split(':')
                key = tmp[0].strip()
                val = tmp[1].strip()
                attr_dict[key] = val
                
            all_keys = all_keys.union(set(attr_dict.keys()))
            for key in all_keys:
                if key not in out_dict:
                    out_dict[key] = ['']*out_len
                if key in attr_dict:
                    out_dict[key].append(attr_dict[key])
                else:
                    out_dict[key].append('')
            
#             print(out_dict)
#             out_df = pd.DataFrame(out_dict)
#             previewdf(out_df)
            out_len += 1
        out_df = pd.DataFrame(out_dict)
        return out_df
    
    def create_table_for_(self, activity):
        print("Creating table for activity:", activity)
        actdf = self.df[self.df.Activity == activity].reset_index(drop=True)

        out_df = self.get_output_df(actdf.Output)
        out_df.loc[:, self.ts] = actdf[self.ts]
        
        # adjust or GMT offset
        out_df[self.gmt_offset] = out_df[self.gmt_offset].apply(lambda x: pd.Timedelta(hours=int(x)))
        out_df.loc[:, self.adjusted_ts] = out_df[self.ts] + out_df[self.gmt_offset]
        
        out_df = out_df.sort_values(self.adjusted_ts)
        print("Done | Shape:", out_df.shape, "| Columns:", out_df.columns)
        previewdf(out_df.tail(5))
        
        return out_df
    
    def get_tables(self):
        for act in self.activities:
            self.tables[act] = self.create_table_for_(act)
        return self.tables
    
cr = Crawler(df)
tables = cr.get_tables()
tables.keys()

Creating table for activity: Knowledge Sync Deletion Bookmark
Done | Shape: (482, 14) | Columns: Index(['END', 'STREAM NAME', 'STRING', 'ZOBJECT TABLE ID', 'USAGE IN MINUTES',
       'ENTRY CREATION', 'START', 'GMT OFFSET', 'USAGE IN SECONDS',
       'DAY OF WEEK', 'UUID', 'VALUE', 'Timestamp', 'AdjustedTimestamp'],
      dtype='object')


,END,STREAM NAME,STRING,ZOBJECT TABLE ID,USAGE IN MINUTES,ENTRY CREATION,START,GMT OFFSET,USAGE IN SECONDS,DAY OF WEEK,UUID,VALUE,Timestamp,AdjustedTimestamp
476,2022-02-23 01,/knowledge-sync-addition-window/4AB7ECFB-3D8E-54C1-A885-9876ECCEFB78,Rapport,955802,1.0000455995400748,2022-02-23 01,2022-02-23 01,-1 days +19:00:00,60.00273597240448,Tuesday,FF3B2B32-3EDC-4183-B7B5-886FE3BBA42F,4.096694909699153e+18,2022-02-23 01:06:43,2022-02-22 20:06:43
478,2022-02-23 01,/knowledge-sync-addition-window/4AB7ECFB-3D8E-54C1-A885-9876ECCEFB78,Rapport,955806,1.001609867811203,2022-02-23 01,2022-02-23 01,-1 days +19:00:00,60.09659206867218,Tuesday,D7144441-0942-40FE-A45F-795C6A0AE2B3,4.096694909699153e+18,2022-02-23 01:07:43,2022-02-22 20:07:43
479,2022-02-23 01,/knowledge-sync-addition-window/531741CF-E6CF-5AD7-8E64-C42D7E532F50,Cloud(Up),955808,1.0059822996457417,2022-02-23 01,2022-02-23 01,-1 days +19:00:00,60.35893797874451,Tuesday,366063CB-59C3-48B9-997C-82829149A4D7,4.625102430714171e+18,2022-02-23 01:07:45,2022-02-22 20:07:45
480,2022-02-23 01,/knowledge-sync-addition-window/4AB7ECFB-3D8E-54C1-A885-9876ECCEFB78,Rapport,955811,1.0000922997792563,2022-02-23 01,2022-02-23 01,-1 days +19:00:00,60.00553798675537,Tuesday,68D2BBBB-D637-45F6-91C7-F283F91013A4,4.096694909699153e+18,2022-02-23 01:08:43,2022-02-22 20:08:43
481,2022-02-23 01,/knowledge-sync-addition-window/531741CF-E6CF-5AD7-8E64-C42D7E532F50,Cloud(Up),955813,1.010808950662613,2022-02-23 01,2022-02-23 01,-1 days +19:00:00,60.648537039756775,Tuesday,D0A41983-1E33-494A-B032-36646F2583D9,4.625102430714171e+18,2022-02-23 01:08:45,2022-02-22 20:08:45


Creating table for activity: Knowledge Sync Addition Window
Done | Shape: (964, 14) | Columns: Index(['END', 'STREAM NAME', 'STRING', 'ZOBJECT TABLE ID', 'USAGE IN MINUTES',
       'ENTRY CREATION', 'START', 'GMT OFFSET', 'USAGE IN SECONDS',
       'DAY OF WEEK', 'UUID', 'VALUE', 'Timestamp', 'AdjustedTimestamp'],
      dtype='object')


,END,STREAM NAME,STRING,ZOBJECT TABLE ID,USAGE IN MINUTES,ENTRY CREATION,START,GMT OFFSET,USAGE IN SECONDS,DAY OF WEEK,UUID,VALUE,Timestamp,AdjustedTimestamp
479,2022-02-23 01,/knowledge-sync-addition-window/531741CF-E6CF-5AD7-8E64-C42D7E532F50,Cloud(Up),955808,1.0059822996457417,2022-02-23 01,2022-02-23 01,-1 days +19:00:00,60.35893797874451,Tuesday,366063CB-59C3-48B9-997C-82829149A4D7,4.625102430714171e+18,2022-02-23 01:07:45,2022-02-22 20:07:45
962,2022-02-23 01,/knowledge-sync-addition-window/4AB7ECFB-3D8E-54C1-A885-9876ECCEFB78,Rapport,955811,1.0000922997792563,2022-02-23 01,2022-02-23 01,-1 days +19:00:00,60.00553798675537,Tuesday,68D2BBBB-D637-45F6-91C7-F283F91013A4,4.096694909699153e+18,2022-02-23 01:08:43,2022-02-22 20:08:43
480,2022-02-23 01,/knowledge-sync-addition-window/4AB7ECFB-3D8E-54C1-A885-9876ECCEFB78,Rapport,955811,1.0000922997792563,2022-02-23 01,2022-02-23 01,-1 days +19:00:00,60.00553798675537,Tuesday,68D2BBBB-D637-45F6-91C7-F283F91013A4,4.096694909699153e+18,2022-02-23 01:08:43,2022-02-22 20:08:43
481,2022-02-23 01,/knowledge-sync-addition-window/531741CF-E6CF-5AD7-8E64-C42D7E532F50,Cloud(Up),955813,1.010808950662613,2022-02-23 01,2022-02-23 01,-1 days +19:00:00,60.648537039756775,Tuesday,D0A41983-1E33-494A-B032-36646F2583D9,4.625102430714171e+18,2022-02-23 01:08:45,2022-02-22 20:08:45
963,2022-02-23 01,/knowledge-sync-addition-window/531741CF-E6CF-5AD7-8E64-C42D7E532F50,Cloud(Up),955813,1.010808950662613,2022-02-23 01,2022-02-23 01,-1 days +19:00:00,60.648537039756775,Tuesday,D0A41983-1E33-494A-B032-36646F2583D9,4.625102430714171e+18,2022-02-23 01:08:45,2022-02-22 20:08:45


Creating table for activity: Application Intents
Done | Shape: (8888, 26) | Columns: Index(['ZMETADATAHASH', 'DIRECTION', 'END', 'ZOBJECT TABLE ID',
       'DERIVED INTENT ID', 'INTENT TYPE', 'APP NAME', 'SOURCE ID',
       'BUNDLE ID', 'NAME', 'GMT OFFSET', 'HANDLING STATUS', 'UUID', 'VALUE',
       'GROUP ID', 'INTENT VERB', 'START', 'DONATED BY SIRI', 'ENTRY CREATION',
       'INTENT CLASS', 'SERIALIZED INTERACTION (HEX)', 'DEVICE ID',
       'DAY OF WEEK', 'ITEM ID', 'Timestamp', 'AdjustedTimestamp'],
      dtype='object')


...

Creating table for activity: Device Discoverability
Done | Shape: (10002, 14) | Columns: Index(['ZMETADATAHASH', 'END', 'ZOBJECT TABLE ID',
       'DISCOVERABILITY SIGNAL CONTEXT', 'DAY OF WEEK', 'ZBUNDLEID',
       'ENTRY CREATION', 'APP NAME', 'START', 'GMT OFFSET', 'OS BUILD', 'UUID',
       'Timestamp', 'AdjustedTimestamp'],
      dtype='object')


,ZMETADATAHASH,END,ZOBJECT TABLE ID,DISCOVERABILITY SIGNAL CONTEXT,DAY OF WEEK,ZBUNDLEID,ENTRY CREATION,APP NAME,START,GMT OFFSET,OS BUILD,UUID,Timestamp,AdjustedTimestamp
9997,197bf0975bcb035e01a0f515ea6985bb,2022-02-22 17,953721,gesture,Tuesday,com.apple.notificationcenterui,2022-02-22 17,com.apple.notificationcenter.opened,2022-02-22 17,-1 days +19:00:00,macOS-21A5506j,82C81915-0F60-40A8-84B5-0DA206D4BF7A,2022-02-22 17:03:15,2022-02-22 12:03:15
9998,197bf0975bcb035e01a0f515ea6985bb,2022-02-22 17,953724,gesture,Tuesday,com.apple.notificationcenterui,2022-02-22 17,com.apple.notificationcenter.opened,2022-02-22 17,-1 days +19:00:00,macOS-21A5506j,741231FF-4DD4-4B85-BD87-66E86C6803BF,2022-02-22 17:03:23,2022-02-22 12:03:23
9999,838592e4698a7babe62d7e660a7c4c50,2022-02-22 17,953791,keyboard.selection,Tuesday,com.apple.screencapture,2022-02-22 17,com.apple.screencapture.invoke,2022-02-22 17,-1 days +19:00:00,macOS-21A5506j,8B5FA486-5BFB-428C-9DCB-B2FFB3E666ED,2022-02-22 17:17:00,2022-02-22 12:17:00
10000,197bf0975bcb035e01a0f515ea6985bb,2022-02-22 23,954321,gesture,Tuesday,com.apple.notificationcenterui,2022-02-22 23,com.apple.notificationcenter.opened,2022-02-22 23,-1 days +19:00:00,macOS-21A5506j,18C3E95A-1439-4030-890D-7DA7E0AABDBD,2022-02-22 23:11:57,2022-02-22 18:11:57
10001,197bf0975bcb035e01a0f515ea6985bb,2022-02-23 00,955677,gesture,Tuesday,com.apple.notificationcenterui,2022-02-23 00,com.apple.notificationcenter.opened,2022-02-23 00,-1 days +19:00:00,macOS-21A5506j,2B021AA1-8A7E-4CD4-AA6E-DEC2B4E9BA4E,2022-02-23 00:37:38,2022-02-22 19:37:38


Creating table for activity: Event Tombstone
Done | Shape: (634, 14) | Columns: Index(['EVENT STREAM NAME', 'ZMETADATAHASH', 'END', 'ZOBJECT TABLE ID',
       'USAGE IN MINUTES', 'ENTRY CREATION', 'UUID', 'START', 'GMT OFFSET',
       'USAGE IN SECONDS', 'DAY OF WEEK', 'EVENT SOURCE DEVICE ID',
       'Timestamp', 'AdjustedTimestamp'],
      dtype='object')


,EVENT STREAM NAME,ZMETADATAHASH,END,ZOBJECT TABLE ID,USAGE IN MINUTES,ENTRY CREATION,UUID,START,GMT OFFSET,USAGE IN SECONDS,DAY OF WEEK,EVENT SOURCE DEVICE ID,Timestamp,AdjustedTimestamp
631,/portrait/topic,223822c84c5c2fb15affc478484792d8,2022-02-22 14,953466,0.0,2022-02-22 14,D9F6E3D4-2B98-48AB-B449-0AFFA6C3A23E,2022-02-22 14,-1 days +19:00:00,0.0,Tuesday,None,2022-02-22 14:20:11,2022-02-22 09:20:11
632,/portrait/topic,223822c84c5c2fb15affc478484792d8,2022-02-22 14,953467,0.0,2022-02-22 14,5A39DAC5-1A7E-402E-A138-5FB2E04EA2E8,2022-02-22 14,-1 days +19:00:00,0.0,Tuesday,None,2022-02-22 14:20:11,2022-02-22 09:20:11
629,/portrait/topic,223822c84c5c2fb15affc478484792d8,2022-02-22 14,953464,0.0,2022-02-22 14,15D4680E-E49F-4F44-B916-D1D4EC82F3BF,2022-02-22 14,-1 days +19:00:00,0.0,Tuesday,None,2022-02-22 14:20:11,2022-02-22 09:20:11
630,/portrait/topic,223822c84c5c2fb15affc478484792d8,2022-02-22 14,953465,0.0,2022-02-22 14,37F26C45-B612-40E2-B218-59652F9496A7,2022-02-22 14,-1 days +19:00:00,0.0,Tuesday,None,2022-02-22 14:20:11,2022-02-22 09:20:11
633,/portrait/topic,223822c84c5c2fb15affc478484792d8,2022-02-22 14,953468,0.0,2022-02-22 14,DDBDEA71-0628-40D9-9271-5C10CDA2EADE,2022-02-22 14,-1 days +19:00:00,0.0,Tuesday,None,2022-02-22 14:20:11,2022-02-22 09:20:11


dict_keys(['Knowledge Sync Deletion Bookmark', 'Knowledge Sync Addition Window', 'Application Intents', 'Now Playing', 'Bluetooth Connected', 'App Media Usage', 'Device Lock Status', 'Portrait Topic', 'Backlight Status', 'Application Web Usage', 'Portrait Entity', 'Notification Usage', 'Application Usage', 'Device Discoverability', 'Event Tombstone'])

In [46]:
webusage_colname = 'Application Web Usage'
appusage_colname = 'Application Usage'
appusagedf = tables[appusage_colname]
webusagedf = tables[webusage_colname]

# filter only times between
start = '02-18-2022 15:25:30'
end = '02-18-2022 20:51:04'
appusage = appusagedf[(appusagedf.AdjustedTimestamp > start) & (appusagedf.AdjustedTimestamp < end)].sort_values('Timestamp')
webusage = webusagedf[(webusagedf.AdjustedTimestamp > start) & (webusagedf.AdjustedTimestamp < end)].sort_values('Timestamp')
print('APP USAGE')
previewdf(appusage.sample(5))
print('WEB USAGE')
previewdf(webusage.sample(5))
print("appusage", appusage.shape, 'columns:', appusage.columns)
print("webusage", webusage.shape, 'columns:', webusage.columns)

APP USAGE


,ZMETADATAHASH,END,ZOBJECT TABLE ID,USAGE IN MINUTES,DEVICE ID (HARDWARE UUID),ENTRY CREATION,BUNDLE ID,NAME,START,GMT OFFSET,USAGE IN SECONDS,DAY OF WEEK,UUID,VALUE,Timestamp,AdjustedTimestamp
6833,bac43d32be11d6a9644b06fab0d861b1,2022-02-19 01,941858,0.08333333333333333,None,2022-02-19 01,com.shahaard.osx.prodczar,None,2022-02-19 01,-1 days +19:00:00,5,Friday,7F919F34-5F8B-4A77-A7DD-33839778AC92,None,2022-02-19 01:50:34,2022-02-18 20:50:34
6598,b1ff6f5fc650baff257fe0c8c33ec346,2022-02-18 21,941119,0.5333333333333333,None,2022-02-18 21,us.zoom.xos,None,2022-02-18 21,-1 days +19:00:00,32,Friday,7D118477-5148-4AEB-89DC-7891A72E17C8,None,2022-02-18 21:09:35,2022-02-18 16:09:35
6835,bac43d32be11d6a9644b06fab0d861b1,2022-02-19 01,941860,0.06666666666666667,None,2022-02-19 01,com.shahaard.osx.prodczar,None,2022-02-19 01,-1 days +19:00:00,4,Friday,1E0A7B91-E7AA-4473-8E98-DE7DCB34D661,None,2022-02-19 01:51:03,2022-02-18 20:51:03
6714,b1ff6f5fc650baff257fe0c8c33ec346,2022-02-18 23,941599,11.483333333333333,None,2022-02-18 23,com.google.Chrome,None,2022-02-18 23,-1 days +19:00:00,689,Friday,BF834F72-E773-4328-BB19-411A99B60646,None,2022-02-18 23:23:12,2022-02-18 18:23:12
6578,b1ff6f5fc650baff257fe0c8c33ec346,2022-02-18 21,941089,0.1,None,2022-02-18 21,notion.id,None,2022-02-18 21,-1 days +19:00:00,6,Friday,0902DC56-7E56-4E3B-86C6-0C2D955749F8,None,2022-02-18 21:03:36,2022-02-18 16:03:36


WEB USAGE


,ZMETADATAHASH,END,ZOBJECT TABLE ID,USAGE IN MINUTES,DEVICE ID (HARDWARE UUID),APP NAME,DIGITAL HEALTH DOMAIN,NAME,GMT OFFSET,USAGE IN SECONDS,UUID,VALUE,ENTRY CREATION,START,DAY OF WEEK,DIGITAL HEALTH URL,DIGITALHEALTH USAGE TYPE,Timestamp,AdjustedTimestamp
8540,c9a7745206290d84b111fd67bfd625b0,2022-02-18 21,941147,0.9833333333333333,None,com.apple.Safari,docs.aws.amazon.com,None,-1 days +19:00:00,59,21AFE401-9BF1-4F52-BD3D-3DE9D5FFED21,None,2022-02-18 21,2022-02-18 21,Friday,https,1,2022-02-18 21:11:47,2022-02-18 16:11:47
8531,3b5c0c97188369b8c004bfe7cc12bd31,2022-02-18 21,941131,0.05,None,com.apple.Safari,rapidminer.com,None,-1 days +19:00:00,3,8A0E32BC-3FAF-41C2-94ED-410A3086E94C,None,2022-02-18 21,2022-02-18 21,Friday,https,1,2022-02-18 21:10:51,2022-02-18 16:10:51
8471,5406d3b21d0ed633ff494281bde6dfd6,2022-02-18 20,940974,0.016666666666666666,None,com.apple.Safari,docs.aws.amazon.com,None,-1 days +19:00:00,1,650906F5-2A20-436F-9333-C690134AB3B2,None,2022-02-18 20,2022-02-18 20,Friday,https,1,2022-02-18 20:41:33,2022-02-18 15:41:33
8514,ee087221336fae6b5f4cd4c9b9a96571,2022-02-18 21,941088,0.1,None,com.apple.Safari,docs.aws.amazon.com,None,-1 days +19:00:00,6,2B611744-A7A0-47DA-B557-486602765538,None,2022-02-18 21,2022-02-18 21,Friday,https,0,2022-02-18 21:03:36,2022-02-18 16:03:36
8701,1723e0a5458bce60a4898e90ae1b3357,2022-02-18 21,941408,0.11666666666666667,None,com.apple.Safari,docs.aws.amazon.com,None,-1 days +19:00:00,7,196F08E6-3112-40AF-9FDF-F6D627458230,None,2022-02-18 21,2022-02-18 21,Friday,https,0,2022-02-18 21:57:57,2022-02-18 16:57:57


appusage (348, 16) columns: Index(['ZMETADATAHASH', 'END', 'ZOBJECT TABLE ID', 'USAGE IN MINUTES',
       'DEVICE ID (HARDWARE UUID)', 'ENTRY CREATION', 'BUNDLE ID', 'NAME',
       'START', 'GMT OFFSET', 'USAGE IN SECONDS', 'DAY OF WEEK', 'UUID',
       'VALUE', 'Timestamp', 'AdjustedTimestamp'],
      dtype='object')
webusage (435, 19) columns: Index(['ZMETADATAHASH', 'END', 'ZOBJECT TABLE ID', 'USAGE IN MINUTES',
       'DEVICE ID (HARDWARE UUID)', 'APP NAME', 'DIGITAL HEALTH DOMAIN',
       'NAME', 'GMT OFFSET', 'USAGE IN SECONDS', 'UUID', 'VALUE',
       'ENTRY CREATION', 'START', 'DAY OF WEEK', 'DIGITAL HEALTH URL',
       'DIGITALHEALTH USAGE TYPE', 'Timestamp', 'AdjustedTimestamp'],
      dtype='object')


In [47]:
# merge both
# first add app / web identifier column
appusage.loc[:, 'Activity'] = appusage_colname
webusage.loc[:, 'Activity'] = webusage_colname
# remove unnecessary columns
appcols = ['Activity', 'AdjustedTimestamp', 'USAGE IN MINUTES', 'BUNDLE ID']
webcols = ['Activity', 'AdjustedTimestamp', 'USAGE IN MINUTES', 'DIGITAL HEALTH DOMAIN', 'APP NAME']
appusage = appusage[appcols].reset_index(drop=True)
webusage = webusage[webcols].reset_index(drop=True)

In [51]:
combined = pd.concat([appusage, webusage]).sort_values('AdjustedTimestamp').reset_index(drop=True)
combined.to_csv('appusage_during_baselines_02-18-21.csv', index=False)
combined

,Activity,AdjustedTimestamp,USAGE IN MINUTES,BUNDLE ID,DIGITAL HEALTH DOMAIN,APP NAME
0,Application Usage,2022-02-18 15:25:32,0.08333333333333333,com.apple.Safari,NaN,NaN
1,Application Web Usage,2022-02-18 15:25:32,0.08333333333333333,NaN,aws.amazon.com,com.apple.Safari
2,Application Usage,2022-02-18 15:25:37,0.15,notion.id,NaN,NaN
3,Application Web Usage,2022-02-18 15:25:37,0.016666666666666666,NaN,aws.amazon.com,com.apple.Safari
4,Application Web Usage,2022-02-18 15:25:39,0.05,NaN,aws.amazon.com,com.apple.Safari
...,...,...,...,...,...,...
778,Application Web Usage,2022-02-18 20:45:41,0.6833333333333333,NaN,us-west-2.console.aws.amazon.com,com.apple.Safari
779,Application Web Usage,2022-02-18 20:46:22,0.016666666666666666,NaN,us-west-2.console.aws.amazon.com,com.apple.Safari
780,Application Usage,2022-02-18 20:50:34,0.08333333333333333,com.shahaard.osx.prodczar,NaN,NaN
781,Application Usage,2022-02-18 20:50:39,0.4,com.apple.Safari,NaN,NaN


## YOLO

In [38]:
yolo = pd.read_csv(datadir('apollo_3.csv'), on_bad_lines='skip', delimiter='\t')
yolo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89109 entries, 0 to 89108
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Timestamp  89109 non-null  object
 1   Activity   89109 non-null  object
 2   Output     89109 non-null  object
 3   Database   89109 non-null  object
 4   Module     89109 non-null  object
dtypes: object(5)
memory usage: 3.4+ MB


In [87]:
yolo_cr = Crawler(yolo)
yolo_tables = yolo_cr.get_tables()
yolo_tables.keys()

Creating table for activity: Knowledge Sync Deletion Bookmark
Done | Shape: (372, 13) | Columns: Index(['START', 'STRING', 'ENTRY CREATION', 'UUID', 'END', 'VALUE',
       'ZOBJECT TABLE ID', 'GMT OFFSET', 'USAGE IN SECONDS',
       'USAGE IN MINUTES', 'DAY OF WEEK', 'STREAM NAME', 'Timestamp'],
      dtype='object')
Creating table for activity: Knowledge Sync Addition Window
Done | Shape: (744, 13) | Columns: Index(['START', 'STRING', 'ENTRY CREATION', 'UUID', 'END', 'VALUE',
       'ZOBJECT TABLE ID', 'GMT OFFSET', 'USAGE IN SECONDS',
       'USAGE IN MINUTES', 'DAY OF WEEK', 'STREAM NAME', 'Timestamp'],
      dtype='object')
Creating table for activity: Application Intents
Done | Shape: (8260, 25) | Columns: Index(['ITEM ID', 'START', 'END', 'GROUP ID', 'BUNDLE ID', 'INTENT TYPE',
       'GMT OFFSET', 'ZOBJECT TABLE ID', 'DAY OF WEEK', 'HANDLING STATUS',
       'DIRECTION', 'UUID', 'DERIVED INTENT ID',
       'SERIALIZED INTERACTION (HEX)', 'DEVICE ID', 'ZMETADATAHASH',
       'APP 

dict_keys(['Knowledge Sync Deletion Bookmark', 'Knowledge Sync Addition Window', 'Application Intents', 'Now Playing', 'Bluetooth Connected', 'App Media Usage', 'Device Lock Status', 'Portrait Topic', 'Backlight Status', 'Application Web Usage', 'Portrait Entity', 'Notification Usage', 'Application Usage', 'Device Discoverability', 'Event Tombstone'])

In [88]:
yolo[yolo.Activity == 'App Media Usage']

,Timestamp,Activity,Output,Database,Module
9416,2022-01-23 20:59:49,App Media Usage,[START: 2022-01-23 20:59:49] [END: 2022-01-23 20:59:54] [USAGE IN SECONDS: 5] [USAGE IN MINUTES: 0.08333333333333333] [BUNDLE ID: com.apple.quicklook.QuickLookUIService] [IS USAGE TRUSTED: 1] [DAY OF WEEK: Sunday] [GMT OFFSET: -5] [ENTRY CREATION: 2022-01-23 20:59:54] [UUID: 7FBBF841-4E9F-4549-B0D5-D7E4360DA844] [ZOBJECT TABLE ID: 877175],/Users/haardshah/Library/Application Support/Knowledge/knowledgeC.db,"modules/knowledge_app_media_usage.txt#knowledgeC.db#SQL Query 14,10.16"
9417,2022-01-23 20:59:57,App Media Usage,[START: 2022-01-23 20:59:57] [END: 2022-01-23 21:00:00] [USAGE IN SECONDS: 3] [USAGE IN MINUTES: 0.05] [BUNDLE ID: com.apple.quicklook.QuickLookUIService] [IS USAGE TRUSTED: 1] [DAY OF WEEK: Sunday] [GMT OFFSET: -5] [ENTRY CREATION: 2022-01-23 21:00:00] [UUID: 74EEB3EB-77F2-4654-9F3F-2E6BAF2114F1] [ZOBJECT TABLE ID: 877176],/Users/haardshah/Library/Application Support/Knowledge/knowledgeC.db,"modules/knowledge_app_media_usage.txt#knowledgeC.db#SQL Query 14,10.16"
9418,2022-01-24 00:56:52,App Media Usage,[START: 2022-01-24 00:56:52] [END: 2022-01-24 00:57:14] [USAGE IN SECONDS: 22] [USAGE IN MINUTES: 0.36666666666666664] [BUNDLE ID: com.apple.Safari] [IS USAGE TRUSTED: 1] [DAY OF WEEK: Sunday] [GMT OFFSET: -5] [ENTRY CREATION: 2022-01-24 00:57:14] [UUID: C170096D-E944-46F8-89E1-067AD722BA3A] [ZOBJECT TABLE ID: 878824],/Users/haardshah/Library/Application Support/Knowledge/knowledgeC.db,"modules/knowledge_app_media_usage.txt#knowledgeC.db#SQL Query 14,10.16"
9419,2022-01-24 04:03:16,App Media Usage,[START: 2022-01-24 04:03:16] [END: 2022-01-24 04:04:19] [USAGE IN SECONDS: 63] [USAGE IN MINUTES: 1.05] [BUNDLE ID: com.apple.Safari] [IS USAGE TRUSTED: 1] [DAY OF WEEK: Sunday] [GMT OFFSET: -5] [ENTRY CREATION: 2022-01-24 04:04:19] [UUID: DB1986FE-DC7D-441D-94D4-679473390ECA] [ZOBJECT TABLE ID: 879458],/Users/haardshah/Library/Application Support/Knowledge/knowledgeC.db,"modules/knowledge_app_media_usage.txt#knowledgeC.db#SQL Query 14,10.16"
9420,2022-01-24 04:06:06,App Media Usage,[START: 2022-01-24 04:06:06] [END: 2022-01-24 04:06:34] [USAGE IN SECONDS: 28] [USAGE IN MINUTES: 0.4666666666666667] [BUNDLE ID: com.apple.Safari] [IS USAGE TRUSTED: 1] [DAY OF WEEK: Sunday] [GMT OFFSET: -5] [ENTRY CREATION: 2022-01-24 04:06:34] [UUID: 6FB421D7-325A-49AD-9C35-C58D27208039] [ZOBJECT TABLE ID: 879464],/Users/haardshah/Library/Application Support/Knowledge/knowledgeC.db,"modules/knowledge_app_media_usage.txt#knowledgeC.db#SQL Query 14,10.16"
...,...,...,...,...,...
9919,2022-02-20 10:55:20,App Media Usage,[START: 2022-02-20 10:55:20] [END: 2022-02-20 10:55:26] [USAGE IN SECONDS: 6] [USAGE IN MINUTES: 0.1] [BUNDLE ID: com.apple.Safari] [IS USAGE TRUSTED: 1] [DAY OF WEEK: Sunday] [GMT OFFSET: -5] [ENTRY CREATION: 2022-02-20 10:55:26] [UUID: E0189EEC-8863-4EB0-B0F1-DD16DD53B883] [ZOBJECT TABLE ID: 944663],/Users/haardshah/Library/Application Support/Knowledge/knowledgeC.db,"modules/knowledge_app_media_usage.txt#knowledgeC.db#SQL Query 14,10.16"
9920,2022-02-20 10:56:48,App Media Usage,[START: 2022-02-20 10:56:48] [END: 2022-02-20 10:56:48] [USAGE IN SECONDS: 0] [USAGE IN MINUTES: 0.0] [BUNDLE ID: com.apple.Safari] [IS USAGE TRUSTED: 1] [DAY OF WEEK: Sunday] [GMT OFFSET: -5] [ENTRY CREATION: 2022-02-20 10:56:48] [UUID: 53B2CE89-6353-4C2C-ABAB-DC8473D58FF6] [ZOBJECT TABLE ID: 944666],/Users/haardshah/Library/Application Support/Knowledge/knowledgeC.db,"modules/knowledge_app_media_usage.txt#knowledgeC.db#SQL Query 14,10.16"
9921,2022-02-20 10:57:14,App Media Usage,[START: 2022-02-20 10:57:14] [END: 2022-02-20 10:57:32] [USAGE IN SECONDS: 18] [USAGE IN MINUTES: 0.3] [BUNDLE ID: com.apple.Safari] [IS USAGE TRUSTED: 1] [DAY OF WEEK: Sunday] [GMT OFFSET: -5] [ENTRY CREATION: 2022-02-20 10:57:32] [UUID: 6402A980-B2E5-4A85-BDBA-822885A749E1] [ZOBJECT TABLE ID: 944667],/Users/haardshah/Library/Application Support/Knowledge/knowledgeC.db,"modules/knowledge_ap

In [89]:
set(yolo_tables.keys()) - set(tables.keys())

{'App Media Usage', 'Device Discoverability'}

In [115]:
yolo_tables['Device Discoverability'].sample(10)

,START,UUID,ENTRY CREATION,OS BUILD,ZMETADATAHASH,END,APP NAME,ZBUNDLEID,GMT OFFSET,DISCOVERABILITY SIGNAL CONTEXT,ZOBJECT TABLE ID,DAY OF WEEK,Timestamp
7268,2021-08-30 15,6FC710CB-78EB-44B9-979C-817747805193,2021-08-30 15,macOS-21A5294g,b53ca87a63a63b83de85a924d9cd2b2e,2021-08-30 15,com.apple.screencapture.invoke,com.apple.screencapture,-4,commandline,640755,Monday,2021-08-30 15:05:47
5590,2021-08-30 13,AF2C32A2-8ED2-4FDB-A7E1-366D8E4D1102,2021-08-30 13,macOS-21A5294g,b53ca87a63a63b83de85a924d9cd2b2e,2021-08-30 13,com.apple.screencapture.invoke,com.apple.screencapture,-4,commandline,639073,Monday,2021-08-30 13:41:26
8716,2021-10-26 21,CEE22262-55D5-4E18-BF3C-E99DE8288BBB,2021-10-26 21,macOS-21A5506j,197bf0975bcb035e01a0f515ea6985bb,2021-10-26 21,com.apple.notificationcenter.opened,com.apple.notificationcenterui,-4,gesture,722354,Tuesday,2021-10-26 21:37:18
8005,2021-09-20 05,C771F933-25D0-4157-A74F-4DC019F6AB6B,2021-09-20 05,macOS-21A5294g,447797f980c6082c8142042f6d38bd0a,2021-09-20 05,com.apple.notificationcenter.opened,com.apple.notificationcenterui,-4,gesture,666677,Monday,2021-09-20 05:28:34
5786,2021-08-30 13,F4C466CC-069F-48CF-ACC0-4FE9A6D3640C,2021-08-30 13,macOS-21A5294g,b53ca87a63a63b83de85a924d9cd2b2e,2021-08-30 13,com.apple.screencapture.invoke,com.apple.screencapture,-4,commandline,639269,Monday,2021-08-30 13:50:50
5497,2021-08-30 13,651C3CB4-B24A-41B8-BEDC-B3E5CD14E2C0,2021-08-30 13,macOS-21A5294g,b53ca87a63a63b83de85a924d9cd2b2e,2021-08-30 13,com.apple.screencapture.invoke,com.apple.screencapture,-4,commandline,638980,Monday,2021-08-30 13:36:31
1094,2021-08-30 09,8F9404A9-C92F-4808-9026-391D9054E021,2021-08-30 09,macOS-21A5294g,b53ca87a63a63b83de85a924d9cd2b2e,2021-08-30 09,com.apple.screencapture.invoke,com.apple.screencapture,-4,commandline,634577,Monday,2021-08-30 09:53:27
2793,2021-08-30 11,853563A9-D4C8-4826-8F2A-1F58FB117359,2021-08-30 11,macOS-21A5294g,b53ca87a63a63b83de85a924d9cd2b2e,2021-08-30 11,com.apple.screencapture.invoke,com.apple.screencapture,-4,commandline,636276,Monday,2021-08-30 11:19:39
4137,2021-08-30 12,4AA999A8-39DA-4FBC-82AC-B616EF4B1C2E,2021-08-30 12,macOS-21A5294g,b53ca87a63a63b83de85a924d9cd2b2e,2021-08-30 12,com.apple.screencapture.invoke,com.apple.screencapture,-4,commandline,637620,Monday,2021-08-30 12:27:43
2633,2021-08-30 11,5ADB1930-6A91-4D11-B9ED-0CBF0BD84B1D,2021-08-30 11,macOS-21A5294g,b53ca87a63a63b83de85a924d9cd2b2e,2021-08-30 11,com.apple.screencapture.invoke,com.apple.screencapture,-4,commandline,636116,Monday,2021-08-30 11:11:32
